In [ ]:
%%capture
!pip install deap scoop tqdm

In [ ]:
# read data
import json
def dump_json(test_data,output_json_name=None):
    if output_json_name is not None:
            with open(output_json_name, "w",encoding='utf8') as output_file:
                json.dump(test_data,output_file,ensure_ascii=False)

def get_json(path):
    with open(path,"r", encoding='utf8') as f:
        data=json.load(f)
    return data

def write_file(content,path):
    with open(path,'w') as f:
        f.write(content)

In [ ]:
total_data = get_json('/content/drive/MyDrive/Academic/IT4663/6_26_new_data.json')+\
        get_json('/content/drive/MyDrive/Academic/IT4663/6_28_impossible_data.json')

In [ ]:
from typing import Dict, List
import numpy as np
# individual representation
# array of m elements, each>0.5 -> visits, else not visits, order of magnitude indicates order of visits
# evaluate fitness

def get_visit_order(ind, data):
    visit = [i for i in range(len(ind)) if ind[i]>data['SURVIVAL_RATE']]
    order = np.argsort(np.array([ind[i] for i in visit]))
    visit = [i+1 for i in visit] # ind start indexs from 0, this convert to 1..m
    visit_order = [0]+[visit[i] for i in order]+[0]
    return visit_order

def evaluate(ind,data):
    item2take, itloc, d = data['item2take'], data['itloc'], data['d']
    visit_order = get_visit_order(ind, data)
    itemcantake = [0]+[sum([itloc[i][j] for j in visit_order[1:-1]]) for i in range(1,len(item2take))]
    satisfy = sum([1 for x,y in zip(itemcantake[1:],item2take[1:]) if x>=y])
    sum_dis = sum([x for y in d for x in y])
    path_dis = sum([d[visit_order[i]][visit_order[i+1]] for i in range(len(visit_order)-1)])
    assert path_dis<sum_dis
    assert satisfy<=len(item2take)-1
    # fitness = num_item_satisfied + path_length / sum(distance_matrix)
    return satisfy - path_dis/sum_dis

def get_optimal(ind, data):
    item2take, itloc, d = data['item2take'], data['itloc'], data['d']
    visit_order = get_visit_order(ind, data)
    itemcantake = [0]+[sum([itloc[i][j] for j in visit_order[1:-1]]) for i in range(1,len(item2take))]
    satisfy = sum([1 for x,y in zip(itemcantake[1:],item2take[1:]) if x>=y])
    if satisfy!=len(item2take)-1:
        return -1
    path_dis = sum([d[visit_order[i]][visit_order[i+1]] for i in range(len(visit_order)-1)])
    return [visit_order, path_dis] # [optimal_path, optimal_value]

def init_pop(pop_size,toolbox,data):
    pop = []
    for _ in range(pop_size):
        tmp_ind = toolbox.individual()
        tmp_ind.fitness.values = [evaluate(tmp_ind,data)]
        pop.append(tmp_ind)
    return pop

In [ ]:
# script
import random,time
import matplotlib.pyplot as plt
import multiprocessing
from deap import creator, base, tools, algorithms

In [ ]:
import time, datetime, copy
from tqdm.auto import tqdm

def ga(params, data):
    POP_SIZE = params['popsize']
    IND_SIZE = data['m']
    SURVIVAL_RATE = params['survival']
    data.update({'SURVIVAL_RATE':SURVIVAL_RATE})
    # set up crossover
    CXPB = params['cxpb']
    # Mutation rate
    MUTPB = params['mut']

    creator.create("UAVFitness", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.UAVFitness)

    toolbox = base.Toolbox()
    toolbox.register("attr_float", random.random)
    toolbox.register("individual", tools.initRepeat, creator.Individual,
                    toolbox.attr_float, n=IND_SIZE)

    # pool = multiprocessing.Pool()
    # toolbox.register("map", pool.map)
    toolbox.register("mate", tools.cxUniform,indpb=0.5)
    toolbox.register("sel",tools.selTournament,tournsize=5)
    toolbox.register("selBest", tools.selBest) 
    toolbox.register("getTopInd",tools.selBest)
    toolbox.register("evaluate", evaluate)

    # print("START!")
    num_generations = params['num_gen']
    max_score=0
    best_gen=0
    max_ind = None
    average_fitness_over_time = list()
    best_fitness_over_time = list()
    population_size_over_time = list()
    num_unique_over_time=list()

    start_time=time.time()
    placeholder = [data]*POP_SIZE
    pop = init_pop(POP_SIZE,toolbox,data)
    offspring=pop
    elites = []
    num_elites = int(POP_SIZE*0.7)
    num_ori = int(POP_SIZE*0.2)
    remant = POP_SIZE - num_elites - num_ori
    for i in tqdm(range(num_generations)):
        # elites = toolbox.selBest(pop,10)
        # type1
        # offspring = list(map(toolbox.clone,toolbox.selBest(pop,80)))+init_pop(20,toolbox,data)
        # type2
        elites = list(map(toolbox.clone,toolbox.selBest(pop+elites,num_elites)))
        offspring = elites + list(map(toolbox.clone,pop[:num_ori]))+init_pop(remant,toolbox,data)

        # elites = toolbox.selBest(pop+elites,20)
        # offspring = toolbox.selBest(pop,95)+init_pop(5,toolbox,data)
        
        # Apply crossover on the offspring
        random.shuffle(offspring)
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        # Apply mutation on the offspring
        for mutant in offspring[:]:
            if random.random() < MUTPB:
                mutant[int(random.random() * IND_SIZE)] = random.random()
                # mutant[int(random.random() * IND_SIZE)] = 0
                del mutant.fitness.values

        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = list(map(toolbox.evaluate, invalid_ind, placeholder[:len(invalid_ind)]))
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = [fit]

        pop=offspring
        # Track average and best fitness
        fitness = 0
        change=False
        for ind in pop:
            fitness += ind.fitness.values[0]
            # save best individual of all time
            if max_score < ind.fitness.values[0]:
                max_score = ind.fitness.values[0]
                if get_optimal(ind,data)!=-1:
                    max_ind = copy.deepcopy(ind)
                    change=True
        if change:
            print("Path of best IND :",get_visit_order(max_ind,data))
            print("Optimal value:",get_optimal(max_ind,data)[1])
            print("Best Fitness :",max_score)
            print('-'*10)
        # print(fitness/POP_SIZE)
        average_fitness_over_time.append(fitness / POP_SIZE)
        best_fitness_over_time.append(max_score)
    time_taken = time.time()-start_time
    if max_ind is None:
        return -1
    return get_optimal(max_ind,data)[0], get_optimal(max_ind,data)[1],time_taken

params = {'popsize':100,
          'survival':0.2,
          'cxpb':0.8,
          'mut':0.2,
          'num_gen':100}
all = {}
for survival in [0.2]:
    params.update({'survival':survival})
    solutions = []
    for i in tqdm(range(len(total_data))):
        print("START!")
        res = ga(params, copy.deepcopy(total_data[i]))
        solutions.append(res)
        dump_json(solutions,'/content/drive/MyDrive/Academic/IT4663/Results/new_ga_100_elite.json')

  0%|          | 0/22 [00:00<?, ?it/s]

START!


/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'UAVFitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


  0%|          | 0/100 [00:00<?, ?it/s]

Path of best IND : [0, 3, 5, 4, 0]
Optimal value: 91
Best Fitness : 9.916819012797076
----------
START!


...

Path of best IND : [0, 484, 573, 523, 547, 5, 530, 512, 268, 566, 44, 339, 513, 330, 379, 386, 516, 107, 89, 127, 13, 102, 447, 12, 367, 359, 347, 454, 17, 455, 228, 9, 563, 10, 182, 458, 46, 235, 133, 408, 101, 146, 23, 509, 208, 169, 322, 71, 98, 318, 200, 237, 485, 289, 500, 6, 111, 250, 278, 27, 140, 542, 445, 18, 492, 467, 320, 474, 540, 92, 240, 239, 360, 104, 396, 592, 110, 517, 526, 36, 226, 187, 84, 598, 62, 518, 591, 153, 157, 478, 179, 112, 197, 178, 25, 443, 555, 388, 457, 227, 264, 338, 533, 553, 142, 31, 590, 230, 137, 305, 194, 337, 236, 201, 293, 281, 548, 562, 164, 30, 284, 521, 451, 465, 67, 79, 596, 115, 156, 88, 20, 259, 383, 368, 441, 232, 260, 126, 504, 245, 210, 266, 514, 534, 580, 161, 401, 90, 554, 196, 277, 593, 100, 582, 346, 66, 423, 2, 415, 174, 175, 22, 39, 195, 438, 257, 91, 131, 74, 327, 85, 241, 381, 118, 130, 219, 238, 335, 253, 340, 114, 476, 267, 304, 355, 149, 285, 150, 180, 4, 141, 433, 382, 402, 459, 391, 222, 83, 41, 206, 7, 29, 358, 223, 470, 13

  0%|          | 0/100 [00:00<?, ?it/s]

Path of best IND : [0, 242, 645, 514, 750, 387, 490, 244, 628, 122, 567, 168, 306, 573, 92, 789, 710, 121, 51, 106, 400, 615, 277, 18, 558, 603, 643, 658, 341, 773, 386, 107, 228, 191, 500, 430, 325, 754, 452, 696, 353, 767, 119, 766, 765, 506, 601, 310, 705, 189, 200, 648, 769, 785, 559, 279, 355, 26, 2, 616, 218, 717, 317, 225, 173, 385, 542, 235, 305, 581, 326, 146, 314, 23, 95, 147, 128, 73, 214, 259, 89, 78, 575, 318, 719, 86, 54, 257, 362, 742, 153, 99, 117, 572, 429, 260, 409, 604, 125, 246, 461, 739, 224, 611, 253, 281, 771, 772, 406, 42, 59, 431, 675, 712, 538, 201, 333, 359, 493, 724, 80, 136, 176, 460, 183, 480, 697, 356, 661, 156, 172, 623, 686, 544, 442, 120, 24, 489, 392, 25, 455, 684, 770, 638, 233, 206, 62, 98, 711, 401, 160, 458, 379, 407, 610, 296, 716, 313, 302, 728, 523, 608, 94, 528, 535, 699, 83, 129, 713, 38, 319, 550, 584, 781, 646, 621, 415, 668, 50, 465, 694, 656, 109, 345, 219, 613, 205, 130, 655, 680, 745, 217, 298, 84, 787, 666, 262, 438, 17, 693, 411, 707,

  0%|          | 0/100 [00:00<?, ?it/s]

Path of best IND : [0, 687, 671, 666, 599, 963, 2, 35, 369, 767, 313, 516, 576, 532, 824, 641, 727, 380, 528, 208, 498, 78, 111, 239, 732, 7, 251, 542, 785, 324, 921, 169, 102, 561, 330, 107, 408, 752, 632, 531, 60, 310, 429, 749, 449, 342, 622, 479, 564, 446, 874, 771, 197, 795, 72, 805, 355, 833, 265, 474, 73, 822, 930, 303, 702, 454, 332, 660, 142, 505, 779, 954, 529, 431, 471, 978, 708, 600, 484, 786, 372, 950, 839, 207, 837, 267, 397, 929, 496, 373, 492, 196, 410, 357, 852, 611, 198, 90, 482, 778, 87, 821, 406, 853, 728, 955, 48, 277, 549, 173, 691, 618, 840, 159, 774, 188, 846, 640, 551, 642, 718, 97, 570, 422, 291, 253, 31, 925, 131, 401, 956, 279, 750, 873, 507, 4, 713, 703, 994, 322, 220, 128, 825, 407, 66, 706, 113, 676, 661, 300, 557, 360, 154, 336, 472, 68, 106, 204, 656, 726, 875, 898, 439, 27, 295, 148, 11, 820, 238, 127, 816, 423, 129, 452, 363, 119, 308, 867, 473, 132, 467, 826, 669, 384, 246, 637, 544, 361, 318, 842, 550, 383, 82, 836, 923, 491, 374, 403, 793, 226, 952